In [1]:
import os
import gc
import json
import pathlib
from datetime import datetime
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import torch
from typing import List, Dict, Union, Tuple, NamedTuple
from tqdm import tqdm
import scml
from scml import pandasx as pdx
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [2]:
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
job_dir = f"models/xgb/{ts}"
pathlib.Path(job_dir).mkdir(parents=True, exist_ok=True)
num_boost_round: int = 100
lr: Tuple[float, float] = (1e-3, 1e-3)
feature_fraction: Tuple[float, float] = (1, 1)
min_data_in_leaf: Tuple[int, int] = (20, 20)
objective: str = "binary:logistic"
n_trials: int = 1
label = "generated"

In [3]:
df = pd.read_parquet("input/features.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221612 entries, 0 to 221611
Columns: 100049 entries, row_id to tf_Ġzhu_ang_zi
dtypes: float32(100033), int16(2), int32(5), int8(1), object(8)
memory usage: 82.6+ GB


In [4]:
features = []
prefixes = ["ch_", "ws_", "ts_", "va_", "tf_"]
for col in df.columns:
    for prefix in prefixes:
        if col.startswith(prefix):
            features.append(col)
features.sort()
print(f"{len(features)} features\n{features[:100]}")

100036 features
['ch_digit_frac', 'ch_len', 'ch_letter_frac', 'ch_punc_frac', 'ch_repeat_char_frac', 'ch_space_frac', 'ch_upper_frac', 'tf_0000_00_Ġ00', 'tf_1_n_1', 'tf_a_Ġand_Ġthe', 'tf_a_Ġet_Ġal', 'tf_a_Ġfranc_a', 'tf_a_Ġin_Ġthe', 'tf_a_Ġis_Ġa', 'tf_a_Ġis_Ġthe', 'tf_a_Ġit_Ġis', 'tf_a_Ġlump_ur', 'tf_a_Ġn_Ġd', 'tf_ab_lan_ca', 'tf_ability_Ġof_Ġthe', 'tf_able_Ġin_Ġthe', 'tf_able_Ġto_Ġthe', 'tf_ac_ro_ix', 'tf_ach_and_ran', 'tf_ach_yc_ardia', 'tf_ad_Ġet_Ġal', 'tf_ag_asc_ar', 'tf_ag_m_ire', 'tf_age_Ġof_Ġthe', 'tf_ah_ont_as', 'tf_ai_Ġet_Ġal', 'tf_ak_Ġet_Ġal', 'tf_al_Ġet_Ġal', 'tf_ality_Ġof_Ġthe', 'tf_all_eng_ed', 'tf_am_Ġet_Ġal', 'tf_amin_oph_en', 'tf_an_Ġet_Ġal', 'tf_ance_Ġof_Ġthe', 'tf_ani_Ġet_Ġal', 'tf_ano_Ġet_Ġal', 'tf_ant_Ġof_Ġthe', 'tf_aph_ra_gm', 'tf_ar_bon_ate', 'tf_ar_Ġet_Ġal', 'tf_ard_Ġet_Ġal', 'tf_are_an_Ġsection', 'tf_ari_Ġet_Ġal', 'tf_aryn_ge_al', 'tf_as_Ġand_Ġthe', 'tf_as_Ġet_Ġal', 'tf_as_Ġin_Ġthe', 'tf_ase_Ġchain_Ġreaction', 'tf_ask_atch_e', 'tf_asm_uch_Ġas', 'tf_atch_e_wan', 

In [5]:
#X_train, X_test, y_train, y_test = train_test_split(tra[features], tra[label], test_size=0.2)

th = 0.29
tra = df[df["white_sim"]>=th].copy()
val = df[df["white_sim"]<th].copy()
t = len(tra)
v = len(val)
n = t+v
print(f"val%={v/n:.4f}, len(tra)={t:,}, len(val)={v:,}")
del df
gc.collect()
dtrain = xgb.DMatrix(tra[features], tra[label], enable_categorical=False)
dval = xgb.DMatrix(val[features], val[label], enable_categorical=False)
pdx.value_counts(val[label])

val%=0.0473, len(tra)=211,120, len(val)=10,492


,count,percent
generated,,
0,10451,0.996092
1,41,0.003908


In [6]:
%%time
model = xgb.train(
   params={
       "objective": objective,
       "learning_rate": 0.2,
       "min_child_weight": 20,
       "colsample_bytree": 0.5,
       "max_depth": 6,
   },
   dtrain=dtrain,
   num_boost_round=2000,
   evals=[(dtrain, "train"), (dval, "val")],
   verbose_eval=40,
   early_stopping_rounds=100,
)
print(f"best score {model.best_score:.5f} at iteration {model.best_iteration}")
model.save_model(f"{job_dir}/model.json")

[0]	train-logloss:0.26067	val-logloss:0.14921
[40]	train-logloss:0.05204	val-logloss:0.01055
[80]	train-logloss:0.03693	val-logloss:0.00881
[120]	train-logloss:0.02921	val-logloss:0.00768
[160]	train-logloss:0.02485	val-logloss:0.00718
[200]	train-logloss:0.02184	val-logloss:0.00666
[240]	train-logloss:0.01969	val-logloss:0.00634
[280]	train-logloss:0.01798	val-logloss:0.00612
[320]	train-logloss:0.01649	val-logloss:0.00603
[360]	train-logloss:0.01538	val-logloss:0.00593
[400]	train-logloss:0.01417	val-logloss:0.00595
[440]	train-logloss:0.01318	val-logloss:0.00611
[453]	train-logloss:0.01289	val-logloss:0.00613
best score 0.00589 at iteration 353
CPU times: user 15h 50min 16s, sys: 2h 27min 15s, total: 18h 17min 32s
Wall time: 1h 46min 8s


In [7]:
%%time
y_true = val[label].tolist()
y_pred = model.predict(data=dval, iteration_range=(0, model.best_iteration+1))
auc = roc_auc_score(y_true, y_pred, average="macro")
print(f"auc={auc:.4f}")
print(f"y_pred={y_pred.shape}\n{y_pred[:5]}")

auc=0.9964
y_pred=(10492,)
[0.49295387 0.00106764 0.00449815 0.16905957 0.81424654]
CPU times: user 2.4 s, sys: 1.39 s, total: 3.79 s
Wall time: 327 ms


In [8]:
%%time
scores = model.get_score(importance_type="gain")
assert len(scores)!=0
rows = []
for feature, score in scores.items():
    rows.append({'importance': score, 'feature': feature})
idf = pd.DataFrame.from_records(rows)
idf = idf.sort_values(["importance"], ascending=False, ignore_index=True)
fp = f"{job_dir}/importance.csv"
idf.to_csv(fp, index=True)
print(f"Saved {fp}")
idf.T.head()

Saved models/xgb/20240122_134738/importance.csv
CPU times: user 204 ms, sys: 365 ms, total: 569 ms
Wall time: 46.2 ms


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665
importance,1215.216797,860.145264,719.439941,643.00531,516.345703,473.394745,467.897888,404.068329,376.327637,298.90918,279.824585,265.950775,246.672729,234.651886,232.335281,215.085403,211.528015,209.306702,205.556412,203.285538,201.448212,200.320511,193.510254,188.975555,185.793243,177.841949,177.603729,175.143234,167.450958,166.416962,160.429489,158.627075,158.339493,156.166031,140.641327,137.300919,130.226669,122.189056,120.889771,120.704376,109.787292,109.477089,108.514145,106.837891,106.508423,106.446983,104.825218,104.257454,104.111549,103.953171,103.831909,100.699013,97.731171,96.817032,91.145454,89.608757,87.136314,86.996231,86.586426,84.676178,84.264709,81.976509,80.776085,78.669968,78.398071,78.078293,76.972191,76.876511,75.178146,74.999687,74.888458,74.884804,72.765305,72.517754,71.627228,71.548172,70.589966,70.332336,69.685875,69.669662,69.480446,69.45784,69.068962,68.40554,66.694107,66.607384,64.709831,64.127983,63.960648,63.111332,62.714081,62.263474,62.01944,61.897106,61.733864,61.554638,59.012428,58.928844,58.285507,57.842545,57.396099,57.272461,56.961018,56.692242,55.956253,54.483711,54.269043,53.688553,52.834248,52.061317,51.951248,51.616154,51.43869,50.885368,50.82143,50.77879,50.561619,49.917145,49.810879,49.627266,49.349216,49.244297,49.100327,48.958801,48.894058,48.802731,48.646736,48.540245,48.195732,48.129124,48.042896,48.003899,46.603077,46.43832,46.162151,45.581978,44.880112,44.811298,44.805096,

In [9]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 1:52:35.642406
